In [171]:
%matplotlib inline
import pandas as pd

import numpy as np
import matplotlib as plt

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

from sklearn import linear_model, model_selection, preprocessing, metrics, datasets
from sklearn.multiclass import OneVsRestClassifier
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score

import re
from datetime import datetime


In [172]:
ecb = pd.read_csv("ECB_Citations.csv")

ecb.dropna()

#ecb['ViolationDate'] = pd.to_datetime(ecb.ViolationDate).dt.date
#ecb = ecb.ix['2016-01-01':'2017-12-01']

ecb = ecb[['CitationNo', 'ViolationDate', 'FineAmount', 'Description', 'Neighborhood', 'CitationStatus', 'Location']]

#Original dataset - https://data.baltimorecity.gov/City-Services/ECB-Citations/ywty-nmtg (size too large)

ecb.shape

(235490, 7)

In [173]:
ecb['ViolationDate'] = pd.to_datetime(ecb.ViolationDate).dt.date
ecb.head()
ecb.dtypes


CitationNo         int64
ViolationDate     object
FineAmount        object
Description       object
Neighborhood      object
CitationStatus    object
Location          object
dtype: object

In [174]:
ecb.to_csv("ECB_Citations.csv")

In [175]:
min_date = datetime_object = datetime.strptime('Jan 1 2017 12:00AM', '%b %d %Y %I:%M%p')
ecb = ecb.loc[ecb['ViolationDate'] > min_date.date()]

ecb

,CitationNo,ViolationDate,FineAmount,Description,Neighborhood,CitationStatus,Location
0,54925748,2017-08-03,$50.00,"EXTERIOR SANITARY MAINTENANCE - TRASH, GARBAGE...",NaN,P,"0000 LIGHT ST\nBaltimore, MD\n"
6,54926332,2017-08-07,$50.00,CONSTRUCTION AND MAINTENANCE OF STORAGE RECEPT...,Winston-Govans,P,"0737 MCCABE AVE\nBaltimore, MD\n(39.352208, -7..."
7,54927058,2017-08-08,$50.00,BULK TRASH ...,McElderry Park,O,"0603 N LAKEWOOD AVE\nBaltimore, MD\n(39.297994..."
8,54926910,2017-07-27,$50.00,UNREGISTERED OR INOPERABLE VEHICLE ...,Cheswolde,P,"6118 BENHURST ROAD\nBaltimore, MD\n(39.364133,..."
14,54927959,2017-08-08,$50.00,HIGH GRASS AND WEEDS ...,Hampden,O,"3517 HICKORY AVE\nBaltimore, MD\n(39.330174, -..."
22,54928536,2017-06-22,$500.00,DUMPING UNDER 25 LBS OF WASTE ON PRIVATE PROPE...,NaN,O,"0000 MYRTLE AVE\nBaltimore, MD\n"
23,54928569,2017-07-02,$1000.00,DUMPING OVER 25 LBS OF WASTE ON PUBLIC PROPERT...,NaN,O,"0000 HOMER AVE\nBaltimore, MD\n"
25,54928809,2017-08-09,$100.00,FAILURE TO FILE A COMPLETED ANNUAL REGISTRATIO...,North Harford Road,O,"2910 BAUERNWOOD AVE\nBaltimore, MD\n(39.367759..."
26,54928775,2017-08-09,$500.00,FAILURE TO FILE ANNUAL VACANT BUILDING REGISTR...,Mosher,O,"2752 W LAFAYETTE AVE\nBaltimore, MD\n(39.29814..."
34,54940481,2017-08-16,$50.00,HIGH GRASS AND WEEDS ...,Hamilton Hills,O,"6906 OLD HARFORD ROAD\nBaltimore, MD\n(39.3684..."


In [176]:
ecb_loc = ecb['Location'].astype(str).tolist()
ecb_loc = list(filter(None, ecb_loc))

ecb_loc = list(map(lambda s: s.replace('\n', ' '), ecb_loc))
re.search('\((.*)\)', '4214 WOODLEA AVE Baltimore, MD (39.333441, -76.555926)').group(1)
type(ecb_loc)
try:
    ecb_loc = list(map(lambda s: re.search('\((.*)\)', s).group(1), ecb_loc))
except:
    pass

ecb_loc

['0000 LIGHT ST Baltimore, MD ',
 '0737 MCCABE AVE Baltimore, MD (39.352208, -76.605525)',
 '0603 N LAKEWOOD AVE Baltimore, MD (39.297994, -76.579392)',
 '6118 BENHURST ROAD Baltimore, MD (39.364133, -76.687633)',
 '3517 HICKORY AVE Baltimore, MD (39.330174, -76.633554)',
 '0000 MYRTLE AVE Baltimore, MD ',
 '0000 HOMER AVE Baltimore, MD ',
 '2910 BAUERNWOOD AVE Baltimore, MD (39.367759, -76.549713)',
 '2752 W LAFAYETTE AVE Baltimore, MD (39.298141, -76.662554)',
 '6906 OLD HARFORD ROAD Baltimore, MD (39.36848, -76.556355)',
 '4337 PLAINFIELD AVE Baltimore, MD (39.325227, -76.557927)',
 '0002 COBBER LANE Baltimore, MD (39.285478, -76.687809)',
 '1339 GORSUCH AVE Baltimore, MD (39.324247, -76.599492)',
 '4920 SCHAUB AVE Baltimore, MD (39.322771, -76.539693)',
 '0000 PACA ST Baltimore, MD ',
 '3324 DUDLEY AVE Baltimore, MD (39.322837, -76.564455)',
 '4913 CHALGROVE AVE Baltimore, MD (39.34889, -76.670092)',
 '4913 CHALGROVE AVE Baltimore, MD (39.34889, -76.670092)',
 '0830 COOKS LANE Balt

In [1]:
ecb_loc = ecb['Location'].astype(str).tolist()
ecb_loc = list(filter(None, ecb_loc))

for i in ecb_loc:
    if (i == None):
        ecb_loc.remove(i)
        
ecb_loc

NameError: name 'ecb' is not defined

In [181]:
ecb_lat = []
ecb_lon = []

ecb_loc = list(map(lambda s: s.replace('\n', ' '), ecb_loc))
ecb_loc

ecb_loc = [str(i) for i in ecb_loc]

ecb_loc

ecb_loc_1 = []

for i in ecb_loc:
    try:
        i = re.search('\((.*)\)', i).group(1)
        ecb_loc_1.append(i)
    except:
        pass
    
ecb_loc_1

['39.352208, -76.605525',
 '39.297994, -76.579392',
 '39.364133, -76.687633',
 '39.330174, -76.633554',
 '39.367759, -76.549713',
 '39.298141, -76.662554',
 '39.36848, -76.556355',
 '39.325227, -76.557927',
 '39.285478, -76.687809',
 '39.324247, -76.599492',
 '39.322771, -76.539693',
 '39.322837, -76.564455',
 '39.34889, -76.670092',
 '39.34889, -76.670092',
 '39.295827, -76.70643',
 '39.273791, -76.691099',
 '39.293737, -76.681222',
 '39.316814, -76.566346',
 '39.354082, -76.559911',
 '39.280842, -76.638436',
 '39.232152, -76.596898',
 '39.289416, -76.647192',
 '39.280842, -76.638436',
 '39.266446, -76.677358',
 '39.28216, -76.694378',
 '39.360197, -76.586904',
 '39.285745, -76.670397',
 '39.340676, -76.550993',
 '39.35663, -76.57968',
 '39.295742, -76.679824',
 '39.308044, -76.59822',
 '39.29134, -76.680413',
 '39.315945, -76.634895',
 '39.285412, -76.590705',
 '39.295386, -76.588031',
 '39.289712, -76.542943',
 '39.30929, -76.57435',
 '39.283177, -76.536783',
 '39.319742, -76.595148

In [182]:
for index in range(len(ecb_loc_1)):
    try:
        ecb_lat.append(float(ecb_loc_1[index].split(",")[0]))
        ecb_lon.append(float(ecb_loc_1[index].split(",")[1]))
    except:
        pass

In [183]:
#Plot permit locations on map of baltimore

import plotly.graph_objs as bp
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

     

data = bp.Data([
    bp.Scattermapbox(
        lat=ecb_lat,
        lon=ecb_lon,
        mode='markers',
        marker=bp.Marker(
            size=14
        ),
        text=['Montreal'],
    )
])

layout = bp.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken='pk.eyJ1IjoicHNpbmdocGF5YWwiLCJhIjoiY2o5MzhtamF5M3RhcTJ4bXJsdnA2Z3ZjbiJ9.lwJJeMpfCBEaxF-A7tRqBw',
        bearing=0,
        center=dict(
            lat=39.286138,
            lon=-76.603388
        ),
        pitch=0,
        zoom=5
    ),
)

fig = dict( data=data, layout=layout )
iplot( fig, validate=False, filename='ECB Citations' )